In [7]:
import networkx as nx
import numpy as np

%matplotlib inline

In [183]:
class politicalGraph(object):
    
    def __init__(self, num_votes = 500, num_evolutions = 10, num_nodes = 435, connectivity = 0.5,
                 conform_pr = 0.5, selection_mech = "stoch_accept"):
        
        self.num_votes = num_votes
        self.num_evolutions = num_evolutions
        self.num_nodes = num_nodes
        self.selection_mech = selection_mech
        G = nx.Graph()
        
        for i in range(self.num_nodes):
            conform = np.random.binomial(1, conform_pr)
            ntype = "conformist" if conform else "idealist"
            G.add_node(i, politician = {"ntype" : ntype, "signal" : "A", "vote" : "A"})
            
        for i in G.nodes():
            for j in G.nodes():
                if i != j:
                    G.add_edge(i, j, weight = np.random.uniform(low = 0.0, high = 1.0))
                    
        self.graph = G
        
    def run(self):
        """Runs defined steps of evolution on the initialized graph."""
        for i in range(self.num_evolutions):
            self._step()
            self._evolve()
    
    def _step(self):
        """One step of voting dymanics excluding the evolution process."""
        for i in range(self.num_votes):
            
            self._signal()
            self._vote()
            self._vote_effect()
    
    def draw(self):
        """Draws current state of the graph."""
        nx.draw_networkx(self.graph)
    
    def _signal(self, prob = 0.5):
        """Round of signaling for nodes, initially happens randomly."""
        for i in self.graph.nodes():
            self.graph.node[i]['politician']['signal'] = "A" if np.random.binomial(1, prob) else "B"
    
    def _calculate_vote(self, node):
        
        A_score = 0
        B_score = 0
        
        for i in self.graph.neighbors(node):
            if self.graph.node[i]['politician']["signal"] == 'A':
                A_score += self.graph[node][i]["weight"]

            else:
                B_score += self.graph[node][i]["weight"]

        return "A" if A_score > B_score else "B"
    
    def _vote(self):
        """Voting in which nodes based on their types do the final decision"""
        for i in self.graph.nodes():
            if self.graph.node[i]['politician']["ntype"] == "idealist":
                self.graph.node[i]['politician']["vote"] = self.graph.node[i]['politician']["signal"]
               
            else:
                self.graph.node[i]['politician']["vote"] = self._calculate_vote(i)
    
    def _vote_effect(self):
        """Calculate the change in relations after the current voting."""
        for edge in self.graph.edges():
            if self.graph.node[edge[0]]['politician']["vote"] == self.graph.node[edge[1]]['politician']["vote"]:
                self.graph[edge[0]][edge[1]]["weight"] = min(1.1 * self.graph[edge[0]][edge[1]]["weight"], 1.0)
            else:
                self.graph[edge[0]][edge[1]]["weight"] *= 0.9
    
    def _fitness(self, node):
        return sum(self.graph.adj[node])
    
    def _evolve(self):
        self._mutate()
        self._select()
    
    def _select(self):
        if self.selection_mech == "stoch_accept":
            
            top = max([self._fitness(node) for node in self.graph.nodes()])
            
            for node in self.graph.nodes():
                if not np.random.binomial(1, self._fitness(node) / top):
                    self.graph.remove_node(node)
    
    def avg_weight(self):
        counter = 0
        sum_weights = 0
        for node in self.graph.nodes():
            if self.graph.has_node(node):
                counter += 1
                sum_weights += sum(self.graph.adj[node]) / len(self.graph.adj[node])
        return sum_weights / counter
    
    def _mutate(self):
        # TODO nodes added later have advantage fix
        conform_pr = sum([1 if self.graph.node[x]['politician']["ntype"] == "conformist" else 0 for x in self.graph.nodes()]) / len(self.graph.nodes())
        
        for i in range(self.num_nodes):
            if not self.graph.has_node(i):
                avg_weight = self.avg_weight()
                conform = np.random.binomial(1, conform_pr)
                ntype = "conformist" if conform else "idealist"
                self.graph.add_node(i, politician = {"ntype" : ntype, "signal" : "A", "vote" : "A"})
                
                for j in self.graph.nodes():
                    if i != j:
                        self.graph.add_edge(i, j, weight = avg_weight)
  

In [184]:
congress = politicalGraph(num_nodes = 20)

In [185]:
congress.run()

In [176]:
congress.graph.nodes()

[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [98]:
g.add_node(1, politician = politician(ntype = "idealist", signal = "A", vote = "B"))

In [101]:
g.node[1]

{'politician': <__main__.politician at 0x7f7175d3a668>}